In [4]:
import numpy as np
from mtalg.random import MultithreadedRNG
import pickle
from tqdm import tqdm
import scipy
import cupy as cp
# import torch
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import multivariate_t
# import Other_Mean_Algs as OMA
import PD_Median_Functions_opt_for_sims

viridis = cm.get_cmap('viridis', 8)

/tmp/ipykernel_3631458/1433574395.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  viridis = cm.get_cmap('viridis', 8)


In [5]:
########### Generate the datasets, unit vectors for the PD median
# ##  and starting values for the MCMC/GD and save in folders
########## This is useful for reproducibility and makes the code slightly faster
#Number of runs in the experiment
num_runs=50
#maximum dimension analysed
d=20
#sample size
ndata=50000
#portion of data contaminated
con_level=0.25


##### First generate the datasets and pickle them #####
# Build a contaminating matrix to add to data to create contaminated datasets
#number of observations contaminated
total=int(con_level*ndata)
# matrix of 5s with total rows
mean_contam=np.full((total,d),5)
# matrix of 0s with n-total rows
mean_clean=np.full((ndata-total,d),0)
# n x d matrix with con_level of rows equal to 5, the rest 0
contam_mat=np.vstack([mean_contam,mean_clean])

# This allows us to set the seed with the mtalg package
mrng = MultithreadedRNG(seed=421, num_threads=10)

#generate the data and write to files
for run in tqdm(range(num_runs)):
    #the clean dataset
    data_clean=mrng.standard_normal(size=(ndata,d))
    #add contamination matrix
    data_con=data_clean+contam_mat
    #now pickle it for reloading later
    #pickle 1
    fn="data/data_set_clean_"+str(run)+".pickle"
    with open(fn,"wb") as file:
        pickle.dump(data_clean,file)
    #pickle 2
    fn="data/data_set_con_"+str(run)+".pickle"
    with open(fn,"wb") as file:
        pickle.dump(data_con,file)


##### Second generate the unit vectors to be used ##### 
#number of unit vectors
nvec=1000
#dimensions in the experiment
dimensions=[2,10,20]

# This allows us to set the seed with the mtalg package
mrng = MultithreadedRNG(seed=422, num_threads=10)
# Generate unit vectors and save them for later
for d in tqdm(dimensions):
    for run in range(num_runs):
        unit_vectors=mrng.standard_normal(size=(nvec,d))
        for i in range(0,nvec):
            unit_vectors[i,:]=unit_vectors[i,:]/np.linalg.norm(unit_vectors[i,:])
    #now pickle it for reloading
        fn="unit_vectors/unit_vectors_dimension_"+str(d)+"_run_"+str(run)+".pickle"
        with open(fn,"wb") as file:
            pickle.dump(unit_vectors,file)



#### Third generate the starting values for langevin and gradient descent #### 

mrng = MultithreadedRNG(seed=423, num_threads=10)
# generate uv and save them for later
for d in tqdm(dimensions):
    for run in range(num_runs):
        start=mrng.standard_normal(size=d)
    # now pickle it for reloading
        fn="starting/starting_dimension_"+str(d)+"_run_"+str(run)+".pickle"
        with open(fn,"wb") as file:
            pickle.dump(start,file)




100%|██████████| 3/3 [00:00<00:00, 10.28it/s]


In [6]:
########### Generate the datasets, unit vectors for the PD median
# ##  and starting values for the MCMC/GD and save in folders
########## This is useful for reproducibility and makes the code slightly faster
#Number of runs in the experiment
num_runs=50
# maximum dimension analysed
d=20
# sample size
ndata=50000


##### First generate the datasets and pickle them #####

# This allows us to set the seed with the mtalg package
mrng = MultithreadedRNG(seed=421, num_threads=10)

#generate the data and write to files
for run in tqdm(range(num_runs)):
    mean = np.full(d,0)    # location
    cov = np.identity(d)  # scale matrix
    data_clean=multivariate_t.rvs(loc=mean, shape=cov, df=1, size=ndata, random_state=421)
    #now pickle it for reloading later
    #pickle 1
    fn="data/data_set_heavy_"+str(run)+".pickle"
    with open(fn,"wb") as file:
        pickle.dump(data_clean,file)



100%|██████████| 50/50 [00:02<00:00, 22.28it/s]
